<a id='introduction'></a>
<h2>Introduction</h2>
<p>
    Today we have so many laptops in the market<br>
    All with different prices, brands, graphic cards, hard drives types and other specs.<br>
    <br>
    we thought to our selves what affects on the price the most. is it the brand? is it the graphics card? or is it a simple thing like the type of RAM or hard drive? 
    <br>
    Also, could we predict the price of a laptop based on it's specs? which elemnts are critical for that prediction?
    <br>
    To answer the following questions we set a goal to our research-  to see if we can build a model that will predict the price of a laptop based of it's specs and also to figure out which of the specs affect the price the most.
</p>

<a id='imports'></a>
<h2 id="imports">Imports</h2>

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from random import randint
import time
from time import sleep
from typing import List

<a id="data_acquisition"></a>
<h2>Data Acquisition</h2>
<p>Considering our options of data acquisition sources, we decided to look for the biggest electronics comparsion websites and scrape data which we thought will be helpful and save it as a dataframe.

The top options we found were:</p>
<ul>
   <li> <a href="https://zap.com/"target="_blank">Zap</a></li>
    <li><a href="https://amazon.com/"target="_blank">Amazon</a></li>
    <li><a href="https://www.ret.co.il/"target="_blank">Ret</a></li>
    <li><a href="https://www.gadgetsnow.com/"target="_blank">gadgetsnow</a></li>
    <li><a href="https://www.wisebuy.co.il/"target="_blank">wisebuy</a></li>
</ul>

<p>After a punctual search in these websites for relevant data and great community scale, we have decided to move on with Zap due to the popularity of the site and amount of features provided for each laptop.</p>




<a id='scraping_challanges'></a>
<h3>Scraping challanges</h3>
<p>While we trying our first attempts to scrape the data from good reads we found out the following issues:<br>
    <ul>
       <li> When sending too many requests in a short period of time, the website started to slow down and the scraping took a long period of time.</li>
        <li>The default user agent used by requests was python: "requests/2.26.0" and we were blocked by Zap after a few requests.</li>
     </ul>
</p>
<p> The way we dealed with those issues was by using sleep and a custom header.

<h3>
Some util functions to help scrape the data:
</h3>
<p>
    <ul>
        <li>sleepms- function made in order to create a delay so the amount of requests sends to the server in parralel will not affect the scraping performance and other users experience in the website</li>
        <li>random_wait- function which is used in order to create random delay so our scraping methodology will not be detected by the servers.</li>
       <li> get_fake_user_agent- function which provides us with a non suspectable user agent.</li>

In [ ]:
def sleepms(milliseconds: int):
    seconds = 0.001 * milliseconds
    sleep(seconds)
    
def random_wait():
    start = time.time()
    sleepms(randint(1, 250))

def get_fake_user_agent():
    return {"User-Agent":"PostmanRuntime/7.29.0"}

In [ ]:
def string_to_number(string: str,replaces: List [str]):
    #to be continued
    # return number_converter("".join(string.split(",")))
    pass

<a id='scraping_process'></a>
<h3>Scraping Process</h3>
<p>
At first glance at the website we thought to ourselves, just how exactly we want to scrape the data?<br>
So went in to the website and started exploring the website in order to find the best route for our scraping proccess.
</p>
<p>
We could clearly see that our best option to get maximum data was in the extra details page which contains huge amount of data.<br>

</p>

<img src="images/extra_details_page.png"/>

</p>
At this point we asked ourselves what is the exact data we wish to scrape?<br>

we decided to go with the following and rename them for the rest of the process:
<ul>
    <li>יצרן - Brand</li>
    <li>תאריך כניסה לזאפ - Zap Enter Date</li>
    <li>התאמה לגיימינג - Gaming Compability</li>
    <li>מערכת הפעלה - Operating System</li>
    <li>תצורת 2 ב- 1 - Two in One</li>
    <li>משקל - Weight</li>
    <li>סדרה - Series</li>
    <li>סוג מעבד - CPU type</li>
    <li>נפח זיכרון RAM - RAM Capacity</li>
    <li>מהירות מעבד - CPU speed</li>
    <li>דגם מעבד - CPU modal</li>
    <li>דור מעבד - CPU generation</li>
    <li>כונן קשיח - Hard drive capacity</li>
    <li>מהירות כונן קשיח - Hard drive Type</li>
    <li>כונן אופטי - Optical Drive</li>
    <li>גודל מסך - Screen Size</li>
    <li>רזולוצית מסך - Screen Resolution</li>
    <li>סוג מסך - Screen Type</li>
    <li>קצב ריענון תצוגה - FPS</li>
    <li>מסך מגע - Touch Screen</li>
    <li>כרטיס מסך - GPU</li>
    <li>מצלמת רשת - Web camera</li>
    <li>אמצעי אבטחה - Secutiry</li>
    <li>חיבורים - Connectors</li>
    <li>רשת אלחוטית - Wifi modal</li>
    <li>מודם סלולרי - Cellular modem</li>
    <li>מחיר מינימלי - Min price</li>
    <li>מחיר מקסימלי - Max price</li>
</ul>
Also, we decided to save the page number in which Zap decided to place the laptop, we figured it has a meaning we could use:
<ul>
    <li>Page Number</li>
</ul>


In [ ]:
def crawl_data(link): 
    page = requests.get(link, headers = get_fake_user_agent())
    soup = BeautifulSoup(page.text, 'html.parser')
    links = soup.findAll ('div', {'class' : 'detailsRow'}, limit=None)
    titles = ['מחיר מינימלי','מחיר מקסימלי']
    spans = soup.find('div','PricesTxt').findAll('span') 
    if len(spans) == 2:
        values = [ spans[1].text, spans[0].text ]
    elif len(spans) == 1: 
        values = [ spans[0].text, spans[0].text ]
    else:
        values = [ None,None ]
    
    for i in range(0, len(links)):
        titles.append(links[i].find('div','detailsRowTitle').text.replace('?','').strip())
        values.append(links[i].find('div','detailsRowTxt').text.strip())


    return dict(zip(titles,values))

In [ ]:
def translate_names(df):
    translated_df = pd.DataFrame()
    translation = {
   "יצרן":"Brand",
   "תאריך כניסה לזאפ":"Zap Enter Date",
   "התאמה לגיימינג":"Gaming Compability",
   "מערכת הפעלה":"Operating System",
   "תצורת ‎ 2 in 1":"Two in One",
   "משקל":"Weight",
   "סדרה":"Series",
   "סוג מעבד":"CPU type",
  "פח זיכרון RAM":"RAM Capacity",
   "מהירות מעבד":"CPU speed",
   "דגם מעבד":"CPU modal",
   "דור מעבד":"CPU generation",
   "כונן קשיח":"Hard drive capacity",
   "מהירות כונן קשיח":"Hard drive Type",
  "כונן אופטי":"Optical Drive",
   "גודל מסך":"Screen Size",
   "רזולוצית מסך":"Screen Resolution",
   "סוג מסך":"Screen Type",
   "קצב ריענון תצוגה":"FPS",
   "מסך מגע":"Touch Screen",
   "כרטיס מסך":"GPU",
   "מצלמת רשת":"Web camera",
   "אמצעי אבטחה":"Secutiry",
   "חיבורים":"Connectors",
   "רשת אלחוטית":"Wifi modal",
   "מודם סלולרי":"Cellular modem",
   "מחיר מינימלי":"Min price",
   "מחיר מקסימלי":"Max price"}

    for key, value in translation.items():
        translated_df[value] = df[key]

    return translated_df

<h3>First Crawl Conclusion - Continue</h3>
<p>
In addition, after further investigating our scrape target website, we see that all genres have up to 25 pages (1250 books) to browse (AT MAX).
Therefore, We are going to run the scrape flow again so we fetch every book we are able to (by selected genres), but this time change the genreSpider class to support offset, so it won't re-read the same books again, AND we will append the books this time to the existing parquet file, instead of overwriting them... other than that, same approach.
NOTE: original Class as been updated.
</p>

<ul>
    <li>
        Same Target Genres, <strong>Altough, only the ones which have any books left to scrape. (the genres which has 1000 books in their df (prev max)</strong>
    </li>
    <li>
        same multi-threades approach
    </li>
</ul>

In [ ]:
def zap_data_crawl_all(BASE_URL,filename,page):
    index = page
    df = pd.DataFrame()
    url = f'{BASE_URL}models.aspx?sog=c:"pclaptop&pageinfo={index}'
    page = requests.get(url,headers = get_fake_user_agent())
    soup = BeautifulSoup(page.text, 'html.parser')
    while soup.select_one('.selectedNumBtn') and soup.select_one('.selectedNumBtn').text == str(index):
        print(f'page number {index}')
        links = soup.findAll ('div', {'class' : 'MoreInfo'}, limit=None)   
        for i in range(0, len(links)):
            link = BASE_URL + links[i].find('a')["href"]
            cd = crawl_data(link)
            cd['מספר עמוד'] = index
            df = pd.concat([df,pd.DataFrame([cd])])
            
        index += 1
        random_wait()
        url = f'{BASE_URL}models.aspx?sog=c:"pclaptop&pageinfo={index}'
        page = requests.get(url,headers = get_fake_user_agent())
        soup = BeautifulSoup(page.text, 'html.parser')
    translated_df = translate_names(df)
    translated_df.to_csv(filename, index=False ,encoding = 'utf:"8:"sig')

In [ ]:
BASE_URL ="https://www.zap.co.il/"
filename = 'test.csv'
page = 1
zap_data_crawl_all(BASE_URL, filename, page)